# **Práctica 11: Random Forest**

## **Predicción de Temperaturas Reales con Random Forest y Árbol de Decisión**

En este análisis, se busca predecir la temperatura real (`actual`) utilizando un conjunto de variables meteorológicas. Para ello, se implementarán y compararán dos modelos: un Árbol de Decisión simple y un Random Forest Regressor, incluyendo una versión optimizada mediante búsqueda de hiperparámetros.

El objetivo es:
- Entrenar modelos de regresión.
- Comparar su desempeño.
- Analizar las variables más influyentes en la predicción.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 

## **Análisis exploratorio inicial (EDA)**

Esto nos permitirá conocer el tipo de variables, valores nulos y posibles errores en los datos.
Se visualiza la distribución y posibles outliers de las variables numéricas con boxplots e histogramas.
Esto ayuda a detectar si es necesario escalar, transformar o eliminar valores extremos.



In [ ]:
# Carga de datos y exploración inicial
temps = pd.read_csv("temps.csv")

print(temps.head())
print(temps.info())
print(temps.describe())
print(temps.shape)

In [ ]:
temps.columns

In [ ]:
# Ver cuantos valores unicos hay en la columna week y que so
temps['week'].unique()

In [ ]:
# Ver cuantos de cada valor hay
temps['week'].value_counts()

In [ ]:
# valores nulos
print(temps.isnull().sum())

In [ ]:
# Eliminar year amtes de dividir
temps = temps.drop(columns=['year'])

In [ ]:
numerical_cols = temps.select_dtypes(include=['float64', 'int64'])
fig = plt.figure(figsize=(10, 10))
sns.boxplot(data=numerical_cols)
plt.title("Boxplot de variables numéricas")
plt.show()

# Análisis de distribución
numerical_cols.hist(figsize=(10, 8), bins=20, color='skyblue', edgecolor='black')
plt.suptitle("Distribución de variables numéricas", fontsize=14)
plt.show()

In [ ]:
print(temps.info())

## **Preparación de Datos**

Se separan las variables predictoras y la variable objetivo, que en este caso es la temperatura real (actual) que queremos predecir.
Se convierten variables categóricas en numéricas mediante one-hot encoding.

In [ ]:
# Separar en variables predictoras y la variable objetivo
X = temps.drop('actual', axis=1)
y = temps['actual']

X = pd.get_dummies(X, dtype=int)
X.head()

## **División en entrenamiento y test**

Se divide el conjunto en un 80% para entrenar y 20% para evaluar, para ver cómo se comporta el modelo con datos no vistos.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [ ]:
X_train.shape , X_test.shape

In [ ]:
y_train.shape , y_test.shape

## **Entrenamiento de modelo base: Random Forest Regressor**

Se entrena un modelo de regresión que utiliza varios árboles para hacer predicciones más precisas y robustas.

In [ ]:
# Entrenamiento de modelo Random Forest 
rforest = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=42)
rforest.fit(X_train, y_train)

y_pred = rforest.predict(X_test)

## **Evaluación del modelo base**

Métricas utilizadas:

MSE (Error cuadrático medio): Penaliza más los errores grandes.

MAE (Error absoluto medio): Promedio de errores absolutos.

R² (Coeficiente de determinación): Mide cuánta varianza de la variable objetivo explica el modelo.
→ Valores cercanos a 1 son buenos.


In [ ]:
# Evaluación del modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nEvaluación del modelo base (Regresión):")
print("Error cuadrático medio (MSE):", mse)
print("Error absoluto medio (MAE):", mae)
print("R²:", r2)

Resultado: R² ≈ 0.82 → El modelo base ya predice bastante bien.

In [ ]:
# importancia de las caracteristicas
importances = rforest.feature_importances_
features = X.columns

indices = np.argsort(importances)

plt.figure(figsize=(10, 6))
plt.title('Importancia de las Características')
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), features[indices], rotation=90)
plt.xlabel('Características')
plt.ylabel('Importancia')
plt.tight_layout()
plt.show()



## **Optimización de hiperparámetros con GridSearchCV**

 Se prueba automáticamente con distintas combinaciones de hiperparámetros para encontrar la que genera mejor rendimiento en validación cruzada.

In [ ]:
# Busqueda en cuadricula de hiperparametro (GridSearchCV)
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42),
                          param_grid,
                           cv=5,
                           scoring="neg_mean_squared_error",
                           n_jobs=-1,
                           verbose=2)

grid_search.fit(X_train, y_train)

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor score(accuracy en validación cruzada):", grid_search.best_score_)

## **Evaluación final con el mejor modelo**

In [ ]:
# Evaluación final con el mejor modelo
best_model = grid_search.best_estimator_

y_pred_best = best_model.predict(X_test)

# Métricas de regresión
mse_best = mean_squared_error(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print("\nEvaluación del modelo optimizado (Regresión):")
print("Error cuadrático medio (MSE):", mse_best)
print("Error absoluto medio (MAE):", mae_best)
print("R²:", r2_best)

Se prueba el modelo optimo con el set de test y se evalúan las métricas como antes.

 Resultado:

MSE bajó un poco

MAE también bajó

R² subió a 0.83

Esto confirma que la optimización fue útil.

## **Comparación con un modelo más simple: Árbol de Decisión**

In [ ]:
# 6. Comparación con un único árbol de decisión
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

y_pred_dt = dt.predict(X_test)

# Métricas de regresión para el árbol de decisión
mse_dt = mean_squared_error(y_test, y_pred_dt)
mae_dt = mean_absolute_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)

print("\nEvaluación del Árbol de Decisión (Regresión):")
print("Error cuadrático medio (MSE):", mse_dt)
print("Error absoluto medio (MAE):", mae_dt)
print("R²:", r2_dt)

Se entrena un solo árbol como referencia.
 Resultado: R² = 0.67, peores errores → Muestra que el Random Forest mejora el rendimiento al combinar múltiples árboles y evitar sobreajuste.

## **Conclusiones**

1. El modelo Random Forest base ya ofrecía un buen rendimiento predictivo, con un R² de 0.82.
2. Tras aplicar GridSearchCV para optimizar los hiperparámetros, se logró mejorar ligeramente el modelo, alcanzando un R² de 0.83.
3. Comparado con un único árbol de decisión (R² de 0.67), el modelo Random Forest demuestra ser más robusto y generaliza mejor los datos.
4. Las métricas de regresión confirman que Random Forest, incluso sin optimización, supera claramente al árbol de decisión individual, tanto en error absoluto como cuadrático.


El uso de Random Forest Regressor ha permitido predecir de forma precisa la temperatura. La comparación con un árbol de decisión individual ha mostrado que el modelo en conjunto generaliza mejor. Además, la optimización de hiperparámetros ha contribuido a una mejora leve en el rendimiento.